In [7]:
!pip install plotly
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
import matplotlib.pyplot as plt
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KDTree

In [8]:
df_april_19 = pd.read_csv('../data/SpotifyAudioFeaturesApril2019.csv')
df_nov_18 = pd.read_csv('../data/SpotifyAudioFeaturesNov2018.csv')

In [9]:
df = pd.concat([df_april_19, df_nov_18], ignore_index=True)
print(df.shape)
assert df.shape[0] == (df_april_19.shape[0] + df_nov_18.shape[0])

(247035, 17)


In [10]:
df = df.drop_duplicates(subset = 'track_id', keep='first')
print(df.shape)

(130989, 17)


In [11]:
df.head(20)

,artist_name,track_id,track_name,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,popularity
0,YG,2RM4jf1Xa9zPgMGRDiht8O,"Big Bank feat. 2 Chainz, Big Sean, Nicki Minaj",0.005820,0.743,238373,0.339,0.000,1,0.0812,-7.678,1,0.4090,203.927,4,0.1180,15
1,YG,1tHDG53xJNGsItRA3vfVgs,BAND DRUM (feat. A$AP Rocky),0.024400,0.846,214800,0.557,0.000,8,0.2860,-7.259,1,0.4570,159.009,4,0.3710,0
2,R3HAB,6Wosx2euFPMT14UXiWudMy,Radio Silence,0.025000,0.603,138913,0.723,0.000,9,0.0824,-5.890,0,0.0454,114.966,4,0.3820,56
3,Chris Cooq,3J2Jpw61sO7l6Hc7qdYV91,Lactose,0.029400,0.800,125381,0.579,0.912,5,0.0994,-12.118,0,0.0701,123.003,4,0.6410,0
4,Chris Cooq,2jbYvQCyPgX3CdmAzeVeuS,Same - Original mix,0.000035,0.783,124016,0.792,0.878,7,0.0332,-10.277,1,0.0661,120.047,4,0.9280,0
5,Curbo,26Y1lX7ZOpw9Ql3gGAlqLK,Debauchery - Original mix,0.001150,0.810,124016,0.417,0.919,9,0.1060,-10.783,0,0.0793,120.025,4,0.8370,0
6,Bingo Play,5eIyK73BrxHLnly4F9PWqg,Grandma - Original mix,0.000539,0.819,132742,0.720,0.863,4,0.0727,-8.895,0,0.1510,124.003,4,0.9340,0
7,G Herbo,13Mf2ZBpfNkgWJowvM5hXh,Bon appétit,0.115000,0.885,181838,0.348,0.000,9,0.1070,-12.569,1,0.4510,142.111,4,0.1800,0
8,34 Feet,7BQaRTHk44DkMhIVNcXy2D,Among - Original mix,0.000058,0.740,124016,0.472,0.847,8,0.0959,-9.008,1,0.0551,120.034,4,0.6220,0
9,Chris Cooq,049RxG2laEl9U1PGYeIqLV,Hazard - Original mix,0.000081,0.813,132742,0.731,0.910,11,0.0727,-8.932,1,0.0697,124.031,4,0.9440,0


## Running the model


In [12]:
number_of_songs = 25
    
df_numerics =  df.drop(columns=['track_id', 'track_name', 'artist_name'])
    
df_scaled = StandardScaler().fit_transform(df_numerics)
df_modeled = KDTree(df_scaled)
       
dist, ind = df_modeled.query(df_scaled, k=(number_of_songs+1))
    
dist_df = pd.DataFrame(dist)
    
scores = (1 - ((dist - dist.min()) / (dist.max() - dist.min()))) * 100
    
columns = ['Searched_Song']
for i in range(number_of_songs):
    columns.append(f'Nearest_Song{i}')

dist_score = pd.DataFrame(scores.tolist(), columns = columns)
    
ind[:(number_of_songs+1)]
    
song_ids = np.array(df.track_id)
    
def find_similars(song_ids, ind):
    similars = []
    for row in ind:
        ids = [song_ids[i] for i in row]
        similars.append(ids)

    return similars 
    
nearest_neighbors = find_similars(song_ids, ind)
    
nearest_neighbors_df = pd.DataFrame(nearest_neighbors, columns=columns)

nearest_neighbors_df.head()
    

/Users/Inceptive/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/data.py:645: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.

/Users/Inceptive/anaconda3/lib/python3.7/site-packages/sklearn/base.py:464: DataConversionWarning:

Data with input dtype int64, float64 were all converted to float64 by StandardScaler.



,Searched_Song,Nearest_Song0,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,...,Nearest_Song15,Nearest_Song16,Nearest_Song17,Nearest_Song18,Nearest_Song19,Nearest_Song20,Nearest_Song21,Nearest_Song22,Nearest_Song23,Nearest_Song24
0,2RM4jf1Xa9zPgMGRDiht8O,2ogGsvbGjJz71Ppc9mTExA,56GWUZPELOP1nK4hQwfVVV,02s1Voowwhr0qTSOrMVEXk,1rG9U7m2wqM0AVmZeIsdtz,3UDOPqusSZZ7NqyWyMMk50,7oDuAatXvqWD8DAa87KRJ2,0MiAP4EQGIuikH23RTyP53,2lD4dQpcac8H2Rq5nttJJh,2MwIkkJujEmXa7o9Nc6HeI,...,7ruuZMhXPNbUOCeOOrgct9,285jcyTcSFIOgpfEgV5aot,53bnLhB1eiqRIGdiBElV2a,58TLHPhZmU9toxhoQbl1VD,1Ov9jGn04J4uhXTq645yIR,1pWeQDyW5ieMmfjr8HP5TU,31cZdDNDDvmPDfBUT1zYFf,4hUXNsZ7z4pVUET7vghMqU,0UGcmjbvLBf7JR4PHntTpd,6bXwuVGSHiJ21EK6RBFlPJ
1,1tHDG53xJNGsItRA3vfVgs,76wqmxKVKXradVxmfGpZfX,0x1MJ34QQHGK9nTmOdSe33,0D8Qknpc7ZtnOU4pEwXGos,3eUyRzsgH4Vjraw3oVqVDg,51jT8cWCrANwOvDkzqKpF4,7vBFNgj6XAYYcp6pKuvjp2,4uO9h2EWJqr1ViWEON957k,1Dfwu69pIy52gVUzbJ6Bwk,1SE0FG6lc3MMZoHj65DTVq,...,4KQt5APGQ8ZAplo8c0grOE,4seJ1XHrowyKXFyUU9Mv2K,0TXNa2fNwtAItS8cfE0tKj,2VFVCndhp8gmtcJ4r6XIRo,1I43nzNJJJhBPclxYledNe,5inSQADrNRh9CfWZq5ARwd,5lLMSFjxdjQso84bMQsbJr,2KBfZWKK8t4e82NtDpIExy,6EB1SjnHKabgFOlU9HAOAN,77URdq8YKypDIwn2RkNsyY
2,6Wosx2euFPMT14UXiWudMy,6MdsMUihzXs34YqYjXy01a,2BNQpd87nSv475zdww9K0Z,54JNbExxKpEXUqeSc63Xwd,6RMMWRA9UY9D8DlPNSUytf,76b1sn3qXpA4PhtSaIHoK7,3z7AceXFwECHGH26FOy3u8,6TXAjof8V0Gr4zFoY00zHR,5puKxbSkahwtI2XJR1xZgr,5hZlzFaN6L8yw1a0VdWpzF,...,0bsEC7UG5TQdaa0ZaWfGlD,4zYMb5KWsNjjobVNdeh4wK,3ggtU1ZOKO8ZNiqPNyXGcm,48jHk4Pkkc5PY7rGdJw66z,6VM9H9tmexHFNLePOXOVPX,6BTra8Fue43w5lS1eEibvo,3ujllSUhcPoWIPmce1RC2n,7hRM2EHbeVZl4fPSK2XxOO,2zTjF6OGxklU5Ulv7iTf4R,0yHbRupKbD9LF7bHWVGyzi
3,3J2Jpw61sO7l6Hc7qdYV91,1jFL774GMnBl5Esa9SN4rL,38oxSM8IHLxt43QV7amf4u,01qc7HHvJdcNviyizhuCSI,17ztPex4W8AT6A6eiNipJ0,4EMfeNehRq5ylzAnjyeyXD,0a7U85BtD3pOKrLMvmoTg0,0xoanCCNnBPaHed0C6VhsV,5KPBYV9FSX88T65iR0zt9a,4Yld8cz0LA6a8n8tEDEsfw,...,2m1nKOPCBqn1SKcURt18ii,2IkMYsKcszarnS0oaok3uX,6P6nxLB49dp0F2xj318319,4VTHITWyaUjHzmwfoXDaB0,5L1xVHzn7IaqwxtmA0Z4pA,7ujF2pwTLGbOAyD69uhefe,5vp6OzDuxX0N0Xng8wk4Lr,6uoCKwyosJfGfDpL3swbG8,0XDWfmaduC6YsUJjJWkd7C,3PKh0SeiiYWYKCTtnr4x9V
4,2jbYvQCyPgX3CdmAzeVeuS,6cjG5e09jR9cmwrNWf33rN,3Sxt87DUcTwIocwQUgnOaB,40En76NSz9PLSXcMMED8OX,3dTwRbYD0xu6F0tDTaQMdF,1aTirUVIiRD28x8KkXtcjx,0ttUWrdZ5gCPyGxulSgL5q,0rNoxCiFdTazhLNv2pHb5c,52zEbsrYSVZ8Vj5XNI8iCg,3HlPpFnbHsnOzrg0X9VOF7,...,3c3i3eQsh6xPp3Le5nwrZG,33NLu0yuNtP6fbfBHUqlJy,6hVb00Lr0iufMl9hEWx5Rw,2PC2XjZqlwCEBzZxOVE6sJ,1R23vfUVYnpQrDuXvRJlai,5ifD4jOkQ5JF95F8kNKikC,7hgJ2PNgNZ3nLLiP3QDOuG,3pTGEo5YPIpv9lAqWXoN1z,56h62SueqtASYGzek13EGd,5QycbVFiAiaJ0zcWy8rdvh


In [13]:
pd.set_option('display.max_columns', None)
nearest_neighbors_df.iloc[[69]]

,Searched_Song,Nearest_Song0,Nearest_Song1,Nearest_Song2,Nearest_Song3,Nearest_Song4,Nearest_Song5,Nearest_Song6,Nearest_Song7,Nearest_Song8,Nearest_Song9,Nearest_Song10,Nearest_Song11,Nearest_Song12,Nearest_Song13,Nearest_Song14,Nearest_Song15,Nearest_Song16,Nearest_Song17,Nearest_Song18,Nearest_Song19,Nearest_Song20,Nearest_Song21,Nearest_Song22,Nearest_Song23,Nearest_Song24
69,3Rx1zM3nDFQzAOYg9Hd0D4,67AHtK2jFq8dlOLSSSbQ7T,2ystp6xiBASPJkFR16gJON,5VNGj3qgKC1n28B9etIoJv,6OarwT6HBT8jW6MsnVwn58,61VbbeUd8bXxzyZYrg4djH,21rvKibsH3WmojUZh5H3Gm,11wxWExHmqBNKIo6zK9NEn,5ZGXAHp0YPYFUMbyMqDQH9,4BMPGmOzi4H3S31B2Ckx0u,1VcVGJ4sqRv2Iruxc8CfYf,1xOoqWTv2wLhUeLtXZTm9q,4SV8h3RlcuQc9jE9MUQfFF,5c1Hz72Bc8VMbghi4MJQus,0iZOviuGDLFc8vSrB4RI2T,7JRV17HtiiXksxDdTdpYTy,7apGuGr4Zf6t9JkATkolAI,0Mw9dLno600aQgA0Gf9Usr,6jUXJaXtxOhBLeWbpR2kN5,1nASmYf1d9HiiIgEOPhYQR,5LAe0lSl7rMle11o6Et9WI,5LZu2syDoQNaA0NptU1YIs,0lz57CGwAyuYdMk7BO72XI,3MDnGMGGC00nbpkLP1r6cN,4QZpmKzjC5t1OxEKCvL7Ft,15sVDXzpwJLfHM99VeP7mR


## Random Sample 3d Scatter Plot

In [21]:
fig = px.scatter_3d(df.sample(n=10000, random_state = 69), x='valence', y='danceability', z='popularity', color='valence', size='popularity',
                    opacity=.7, hover_name='track_name', color_discrete_sequence=px.colors.sequential.Plasma[-2::-1],
                    template="plotly_dark") #text = 'artist_name')
plot(fig)

'temp-plot.html'

## Scatter Plot of 25 Nearest Songs

In [26]:
a = df.query("track_name == 'Prop - Original mix'").index.values


In [31]:
a

array([18])

In [32]:
# TO  display specific Row
id_numbers = nearest_neighbors_df.iloc[[a[0]]].to_string(header=False, index=False, index_names=False).split()
        
to_be_3d = df.query('track_id == @id_numbers')


fig = px.scatter_3d(to_be_3d, x='valence', y='danceability', z='popularity', color='valence', size='popularity',
                    opacity=.7, hover_name='track_name', color_discrete_sequence=px.colors.sequential.Plasma[-2::-1],
                    template="plotly_dark", text = 'artist_name')
plot(fig)

#import plotly.io as pio
#py.plot(fig, filename='name.html')

'temp-plot.html'